In [70]:
import csv
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.formula.api as smf
import os
from datetime import datetime

%matplotlib inline

# Functions

In [71]:
def import_dict(dict_loc):
    """ Loads a dictionary from a csv """
    df = pd.read_csv(dict_loc)
    df.index = df.iloc[:,0]
    df = df.drop(df.columns[0], axis = 1)
    return df.to_dict()

def convertDateToQuarter(date):
    quarter = (date.month-1)//3 + 1
    return (str(date.year) + 'Q' + str(quarter))

# Data Import

## Directories

In [72]:
# 605 and 606 csv directory
dir_605 = '../data/605/'
dir_606 = '../data/606/'

## Raw Data

In [73]:
## Import 605 data
# Find market center csvs
marketcenter_csv_list = [x for x in os.listdir(dir_605) if x[-4:] == '.csv']
# Get .csv directories
marketcenter_csv_dirs = [dir_605 + x for x in marketcenter_csv_list]
# Read .csv files
csv_df_list_605 = [pd.read_csv(file, sep = ',') for file in marketcenter_csv_dirs]
# Merge each marketcenter's data
rawdata_605 = pd.concat(csv_df_list_605)
# Clean up
del(csv_df_list_605)

In [74]:
## Import 606 Data

csv_df_list_606 = []
# Find broker folders
broker_folders = [x for x in os.listdir(dir_606) if '.' not in x]
# Merge .csv's for each broker
for broker in broker_folders:
    # Get file locations of csv's for each broker
    directory = dir_606 + broker
    broker_csv_list  = [x for x in os.listdir(directory)]
    broker_csv_dirs  = [dir_606 + broker + '/' + csv for csv in broker_csv_list]
    # Read csv's as dataframes
    csv_df_list_606_broker = [pd.read_csv(file) for file in broker_csv_dirs]
    csv_df_list_606.append(pd.concat(csv_df_list_606_broker))
    
# Merge each broker's data
rawdata_606 = pd.concat(csv_df_list_606)
# Clean up
del(csv_df_list_606)

## Dictionaries

In [75]:
symbol_dict = import_dict('../data/keys/symbols.csv')['Exchange']
mktctr_mpid_dict = import_dict('../data/keys/mpids.csv')['MPID']
ordertype_dict = {11: 'Market', 12: 'Limit'}
broker_vol_dict = import_dict('../data/keys/broker_volumes.csv')

# Data Prep

## Broker Data

### Prepare Raw Data

In [76]:
# Import Data
data_606 = rawdata_606.copy()

# Fix Routing Venue labels
data_606['RoutingVenue'] = data_606['RoutingVenue'].apply(
    lambda x: mktctr_mpid_dict.get(x.strip(), "(Unknown) " + str(x.strip())))

# Drop unknown routing venues
data_606 = data_606[data_606['RoutingVenue'].apply(lambda x: not x.startswith('(Unk'))]

# Convert date to quarter
data_606['Quarter'] = data_606['Date'].apply(
    lambda x: convertDateToQuarter(datetime.strptime(str(x), '%Y%m')))
data_606['Quarter'] = pd.PeriodIndex(data_606['Quarter'], freq='Q').values
data_606 = data_606.drop('Date', axis=1)

# Change column names
data_606 = data_606.rename(
    columns={'RoutingVenue': 'MarketCenter', 'Pct': 'MktShare'})

# Merge known marketcenters of same firm
data_606 = data_606.groupby(['Broker', 'Exchange', 'OrderType', 'Quarter', 'Rebate', 'MarketCenter']).sum().reset_index()

# Add binary var for presence of rebates
data_606['Rebate_Dummy'] = (data_606['Rebate'].apply(lambda x: (x > 0))
                            | data_606['Broker'].apply(lambda x: x == 'TD Ameritrade')).apply(lambda x: int(x))

# Filter 606 data to market centers with data available
mktctrs_available = rawdata_605['MarketCenter'].unique()
data_606 = data_606[data_606['MarketCenter'].apply(lambda x: x in mktctrs_available)]

### Fill in missing 0's

In [77]:
data_606['Obs_id'] = data_606['Broker'] + '-' + data_606['MarketCenter'] + '-' + data_606['Exchange'] + '-' + data_606['OrderType']
data_606_new = data_606.copy()

dates_set = pd.Series(list(data_606['Quarter'].unique())).sort_values()

rebate_dummy_dict = {broker: data_606.query('Broker == "' + broker + '"').iloc[0]['Rebate_Dummy'] for broker in data_606['Broker'].unique()}

# from second element onwards
for quarter in dates_set.iloc[0:]: 
    
    print(quarter, end = ' ')
    mask_1 = (data_606['Quarter'] <  quarter) & (data_606['Quarter'] >= (quarter - 2)) # within last given period
    mask_2 = (data_606['Quarter'] == quarter)
    
#     print('Elements in previous quarters')
#     display(data_606.loc[mask_1].sort_values(by = 'Quarter'))
    
#     print('Elements in this quarter')
#     display(data_606.loc[mask_2])
            
    set_1 = set(data_606.loc[mask_1]['Obs_id'].unique())
    set_2 = set(data_606.loc[mask_2]['Obs_id'].unique())
    
#     display(set_2 - set_1)
    
    # missing id's (last period obs that are not in this period)
    id_list = [list(x.split('-')) for x in (set_1 - set_2)]    
#     display(id_list)
    
    # add missing id's
    print('(%d)' % len(id_list), end = ', ')
    for missing_id in id_list:
        #print(missing_id)
        
        data_606_new = data_606_new.append({'Broker': missing_id[0], 'MarketCenter': missing_id[1], 'Exchange': missing_id[2], 
                            'OrderType': missing_id[3], 'Quarter': quarter, 'Obs_id': '-'.join(missing_id),
                            'Rebate_Dummy': rebate_dummy_dict.get(missing_id[0], np.nan), 'MktShare': 0}, 
                           ignore_index = True)
    
#     print('Updated for this quarter')
#     display(data_606.loc[(data_606['Quarter'] == quarter)])

data_606 = data_606_new.copy()
data_606.head()

2010Q1 (0), 2010Q2 (4), 2010Q3 (0), 2010Q4 (0), 2011Q1 (0), 2011Q2 (4), 2011Q3 (4), 2011Q4 (8), 2012Q1 (8), 2012Q2 (12), 2012Q3 (16), 2012Q4 (12), 2013Q1 (4), 2013Q2 (4), 2013Q3 (0), 2013Q4 (4), 2014Q1 (4), 2014Q2 (40), 2014Q3 (60), 2014Q4 (56), 2015Q1 (104), 2015Q2 (176), 2015Q3 (104), 2015Q4 (84), 2016Q1 (136), 2016Q2 (184), 2016Q3 (692), 2016Q4 (532), 2017Q1 (84), 2017Q2 (240), 2017Q3 (220), 2017Q4 (920), 

,Broker,Exchange,OrderType,Quarter,Rebate,MarketCenter,MktShare,Rebate_Dummy,Obs_id
0,AXA,NASDAQ,Limit,2014Q1,0.0,CDRG,0.1997,0,AXA-CDRG-NASDAQ-Limit
1,AXA,NASDAQ,Limit,2014Q1,0.0,FBCO,0.1205,0,AXA-FBCO-NASDAQ-Limit
2,AXA,NASDAQ,Limit,2014Q1,0.0,G1ES,0.0293,0,AXA-G1ES-NASDAQ-Limit
3,AXA,NASDAQ,Limit,2014Q2,0.0,CDRG,0.1906,0,AXA-CDRG-NASDAQ-Limit
4,AXA,NASDAQ,Limit,2014Q2,0.0,FBCO,0.1436,0,AXA-FBCO-NASDAQ-Limit


## Market Center Data

In [78]:
# Import data
data_605 = rawdata_605.copy()

# Quarter column
data_605['Quarter'] = data_605['idate'].apply(lambda x: convertDateToQuarter(datetime.strptime(str(x), '%Y%m')))
data_605['Quarter'] = pd.PeriodIndex(data_605['Quarter'], freq='Q').values
data_605 = data_605.drop('idate', axis = 1)

# Temporary Variables for Aggregation
data_605['PrImp_TotalT']     = data_605['PrImpShares']    * data_605['PrImp_AvgT']
data_605['PrImp_TotalAmt']   = data_605['PrImpShares']    * data_605['PrImp_AvgAmt']
data_605['ATQ_TotalT']       = data_605['ATQShares']      * data_605['ATQ_AvgT']
data_605['OTQ_TotalT']       = data_605['OTQShares']      * data_605['OTQ_AvgT']
data_605['AvgRealSpread_T']  = data_605['AvgRealSpread']  * data_605['ExecShares']
data_605['AvgEffecSpread_T'] = data_605['AvgEffecSpread'] * data_605['ExecShares']

data_605 = data_605.groupby(['MarketCenter', 'Quarter', 'Exchange', 'OrderCode']) \
        .sum().reset_index()

# Reconstruct original variables
data_605['PrImp_AvgT']     = data_605['PrImp_TotalT']     / data_605['PrImpShares']
data_605['PrImp_AvgAmt']   = data_605['PrImp_TotalAmt']   / data_605['PrImpShares'] 
data_605['ATQ_AvgT']       = data_605['ATQ_TotalT']       / data_605['ATQShares']
data_605['OTQ_AvgT']       = data_605['OTQ_TotalT']       / data_605['OTQShares']
data_605['AvgRealSpread']  = data_605['AvgRealSpread_T']  / data_605['ExecShares']
data_605['AvgEffecSpread'] = data_605['AvgEffecSpread_T'] / data_605['ExecShares'] 
data_605['PrImp_Pct']      = data_605['PrImpShares']      / data_605['ExecShares']
data_605['ATQ_Pct']        = data_605['ATQShares']        / data_605['ExecShares']
data_605['OTQ_Pct']        = data_605['OTQShares']        / data_605['ExecShares']

## New Vars

# Absolute
data_605['OrderType']    = data_605['OrderCode'].apply(lambda x: ordertype_dict.get(x, 'Other'))
data_605['PrImp_ExpAmt'] = data_605['PrImp_AvgAmt'] * data_605['PrImp_Pct']
data_605['All_AvgT']     = (data_605['PrImp_TotalT'] + data_605['ATQ_TotalT'] + data_605['OTQ_TotalT']) \
                            / data_605['ExecShares']
data_605 = data_605.rename(columns = {'idate': 'Date'})

# Relative values
data_605_grouped = data_605.groupby(['Exchange', 'OrderType', 'Quarter'])

data_605['MktCtrAvg_PrImp_Pct']  = data_605_grouped['PrImp_Pct'].transform("mean")
data_605['Rel_PrImp_Pct']        = data_605['PrImp_Pct'] - data_605['MktCtrAvg_PrImp_Pct']
data_605['MktCtrAvg_PrImp_AvgT'] = data_605_grouped['PrImp_AvgT'].transform("mean")
data_605['Rel_PrImp_AvgT']       = data_605['PrImp_AvgT'] - data_605['MktCtrAvg_PrImp_AvgT']    
data_605['MktCtrAvg_PrImp_ExpAmt'] = data_605_grouped['PrImp_ExpAmt'].transform("mean")
data_605['Rel_PrImp_ExpAmt']       = data_605['PrImp_ExpAmt'] - data_605['MktCtrAvg_PrImp_ExpAmt']   
data_605['MktCtrAvg_All_AvgT'] = data_605_grouped['All_AvgT'].transform("mean")
data_605['Rel_All_AvgT']       = data_605['All_AvgT'] - data_605['MktCtrAvg_All_AvgT']   

data_605.head()

,MarketCenter,Quarter,Exchange,OrderCode,CoveredOrders,CoveredShares,CancelledShares,MktCtrExecShares,AwayExecShares,ExecShares_0_9,...,PrImp_ExpAmt,All_AvgT,MktCtrAvg_PrImp_Pct,Rel_PrImp_Pct,MktCtrAvg_PrImp_AvgT,Rel_PrImp_AvgT,MktCtrAvg_PrImp_ExpAmt,Rel_PrImp_ExpAmt,MktCtrAvg_All_AvgT,Rel_All_AvgT
0,AQUA,2010Q1,NASDAQ,11,72,367000,113800,250900,0,250900,...,0.010296,0.065564,0.557361,0.442639,0.357667,-0.292103,0.003442,0.006854,0.714655,-0.649091
1,AQUA,2010Q1,NASDAQ,12,43168,172330609,171680431,321000,49778,227200,...,0.009834,205.496942,0.405479,0.542231,34.086084,132.742186,0.002626,0.007208,84.459308,121.037634
2,AQUA,2010Q1,NASDAQ,13,20391,68389031,68061331,110900,9700,73000,...,NaN,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,0.000000,0.000000
3,AQUA,2010Q1,NASDAQ,14,13031,47528964,47400706,0,8158,0,...,NaN,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,0.000000,0.000000
4,AQUA,2010Q1,NASDAQ,15,19537,74494919,74369968,0,9869,0,...,NaN,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,0.000000,0.000000


# Merge Datasets

In [87]:
data_merged = data_605.merge(data_606)

data_merged = data_merged.query('OrderCode < 13')
data_merged = data_merged.drop('Obs_id', axis = 1)
data_merged.set_index(['Quarter'])
data_merged['Broker_Size'] = data_merged['Broker'].apply(lambda x: broker_vol_dict['Size'].get(x))


print('Total Observations: ' + str(len(data_merged)))
print('Brokers: ' + str(len(set(list(data_merged['Broker'])))))
print('Market Centers: ' + str(len(set(list(data_merged['MarketCenter'])))))

data_merged.head()

Total Observations: 6817
Brokers: 32
Market Centers: 12


,MarketCenter,Quarter,Exchange,OrderCode,CoveredOrders,CoveredShares,CancelledShares,MktCtrExecShares,AwayExecShares,ExecShares_0_9,...,Rel_PrImp_AvgT,MktCtrAvg_PrImp_ExpAmt,Rel_PrImp_ExpAmt,MktCtrAvg_All_AvgT,Rel_All_AvgT,Broker,Rebate,MktShare,Rebate_Dummy,Broker_Size
0,AQUA,2015Q4,Other,11,1,5000,5000,0,0,0,...,NaN,0.004768,NaN,20.208101,NaN,Deutsche,1.0,0.0,1,2.045009e+10
1,AQUA,2015Q4,Other,12,96,375200,223200,0,0,0,...,NaN,0.001124,NaN,19.623809,NaN,Deutsche,1.0,0.0,1,2.045009e+10
5,AQUA,2016Q1,Other,11,1,3000,3000,0,0,0,...,NaN,0.006184,NaN,21.218073,NaN,Deutsche,NaN,0.0,1,2.045009e+10
6,AQUA,2016Q1,Other,12,108,457600,202800,0,0,0,...,NaN,0.001188,NaN,15.650414,NaN,Deutsche,NaN,0.0,1,2.045009e+10
10,AQUA,2016Q2,NASDAQ,11,92,465340,158940,303400,0,296000,...,0.313173,0.010230,0.0196,0.200564,0.300425,Credit Suisse,1.0,0.0,1,3.266721e+10


# Data Export

## Panel

In [88]:
# test
data_merged[data_merged['Quarter'] == pd.Period('2017Q1')].query('Broker == "Deutsche" & OrderType == "Market" & Exchange == "NASDAQ"')

,MarketCenter,Quarter,Exchange,OrderCode,CoveredOrders,CoveredShares,CancelledShares,MktCtrExecShares,AwayExecShares,ExecShares_0_9,...,Rel_PrImp_AvgT,MktCtrAvg_PrImp_ExpAmt,Rel_PrImp_ExpAmt,MktCtrAvg_All_AvgT,Rel_All_AvgT,Broker,Rebate,MktShare,Rebate_Dummy,Broker_Size
507,ARCA,2017Q1,NASDAQ,11,313425,185419644,58836,108941685,76417173,185357658,...,-0.113232,0.006163,-0.004331,0.211129,-0.208189,Deutsche,1.0,0.0002,1,2.045009e+10
5324,CITI,2017Q1,NASDAQ,11,6,11095,9802,0,0,0,...,NaN,0.006163,NaN,0.211129,NaN,Deutsche,1.0,0.0000,1,2.045009e+10
5545,EDGX,2017Q1,NASDAQ,11,181386,112333278,87923121,17092440,7317717,24410157,...,-0.118755,0.006163,-0.003969,0.211129,-0.207215,Deutsche,1.0,0.0000,1,2.045009e+10
7911,FBCO,2017Q1,NASDAQ,11,45276,47036745,80061,436911,46519773,46814277,...,0.088561,0.006163,-0.003845,0.211129,0.099879,Deutsche,1.0,0.0000,1,2.045009e+10
15298,UBSS,2017Q1,NASDAQ,11,2822466,2096918901,2536008,1409946276,671840481,2079458778,...,0.142654,0.006163,0.002769,0.211129,0.462117,Deutsche,1.0,0.0016,1,2.045009e+10


## Fixed Effects

In [89]:
data_merged_demeaned = data_merged - data_merged.groupby(
    ['Broker', 'MarketCenter', 'Exchange', 'OrderType']).transform("mean")

data_merged_demeaned[['Broker', 'Exchange', 'MarketCenter', 'OrderType', 'Quarter']] = data_merged[['Broker', 'Exchange', 'MarketCenter', 'OrderType', 'Quarter']]

data_merged_demeaned['Rebate_Dummy'] = data_merged['Rebate_Dummy']

## To CSV

In [90]:
# Panel
data_merged.to_csv('../data/processed/regression_data_levels.csv', index=False)

# Demeaned
data_merged_demeaned.to_csv(
    '../data/processed/regression_data_levels_demeaned.csv', index=False)

# 605 and 606
data_605.to_csv('../data/processed/605_processed.csv', index = False)
data_606.to_csv('../data/processed/606_processed.csv', index = False)

# raw
rawdata_605.to_csv('../data/605/rawdata_605.csv', index = False)
rawdata_606.to_csv('../data/606/rawdata_606.csv', index = False)

In [103]:
data_merged_demeaned.query('Broker == "TD Ameritrade" & OrderType == "Market"')[['PrImp_Pct', 'PrImp_AvgAmt','PrImp_ExpAmt', 
     'PrImp_AvgT', 'All_AvgT']].dropna()

,PrImp_Pct,PrImp_AvgAmt,PrImp_ExpAmt,PrImp_AvgT,All_AvgT
642,0.000000,0.000000,0.000000,0.000000,0.000000
3425,0.008185,-0.001343,-0.001055,0.029496,0.050452
3534,-0.003933,-0.000104,-0.000119,-0.000588,-0.000424
3642,0.006590,0.000120,0.000154,-0.010298,-0.022134
3760,0.012935,-0.000466,-0.000251,-0.011138,-0.016358
3879,0.010709,-0.000741,-0.000593,-0.000105,-0.000671
3988,0.005276,-0.000083,-0.000036,0.006424,-0.010257
4111,-0.004480,0.001010,0.000814,-0.009637,-0.016202
4230,-0.001511,0.000143,0.000120,-0.000131,0.000729
4345,-0.016090,0.000382,0.000225,-0.007503,-0.018689
